In [2]:
import argparse
import datetime
from pathlib import Path
import shutil

import fastai
import pandas as pd
import pymongo
import sacred
import sklearn.metrics
from fastai.basic_train import get_preds
from fastai.callbacks import *
from fastai.datasets import *
from fastai.imports import nn, torch
from fastai.metrics import *
from fastai.text import *
from fastai.text.data import DataBunch
from fastai.train import *
from fastai.vision import *
from sacred import Experiment
from sacred.observers import MongoObserver
from sklearn import metrics

import news_utils.fastai

In [3]:
UT = Path('~/data/ynacc_proc/proper_threads/data/cls/only_threads_unlimited_30000')

In [5]:
df = pd.read_csv(UT/'train.csv')

In [15]:
df_o = df.groupby('sdid').last()[['text_proc', 'constructiveclass']]

df_o['constructiveclass'] = df_o['constructiveclass'].apply(lambda x: 1 if x == 'Constructive' else 0)

In [17]:
df_o

,text_proc,constructiveclass
sdid,,
27,"xx_thread_start xx_comment_start She was 00, n...",1
44,xx_thread_start xx_comment_start There is no p...,1
80,xx_thread_start xx_comment_start I guess the l...,1
219,xx_thread_start xx_comment_start You could arg...,0
288,xx_thread_start xx_comment_start To the Americ...,0
442,xx_thread_start xx_comment_start When I was in...,1
523,xx_thread_start xx_comment_start There was a s...,0
729,xx_thread_start xx_comment_start I have to agr...,1
847,xx_thread_start xx_comment_start Chinese are s...,1
